In [1]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [1]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [4]:
# This is just a test
df = pd.DataFrame()
fixture = 'fixture.json'
team_home = dirName + '720/720.json'
team_away = dirName + '529/529.json'

with open(fixture) as f:
    my_fixture = json.load(f)
with open(team_home) as f:
    standings_home = json.load(f)
    del standings_home['stats_away']
    my_fixture.update({"team_home": standings_home})
with open(team_away) as f:
    standings_away = json.load(f)
    del standings_home['stats_home']
    my_fixture.update({"team_away": standings_away})
df = df.append(json_normalize(my_fixture))
df.head()

NameError: name 'dirName' is not defined

In [16]:
for col in df.columns: 
    print(col)

goals_home
goals_away
team_home.team.id
team_home.team.name
team_home.rank
team_home.points
team_home.goals_diff
team_home.form
team_home.description
team_away.team.id
team_away.team.name
team_away.rank
team_away.points
team_away.goals_diff
team_away.form
team_away.description
team_away.stats_home.played
team_away.stats_home.wins
team_away.stats_home.draws
team_away.stats_home.lose
team_away.stats_home.goals_for
team_away.stats_home.avg_goals_for
team_away.stats_home.goals_against
team_away.stats_home.avg_goals_against
team_away.stats_home.clean_sheet
team_away.stats_home.failed_to_score
team_away.stats_home.streaks.wins
team_away.stats_home.streaks.draws
team_away.stats_home.streaks.loses
team_away.stats_home.streaks.best_win
team_away.stats_home.streaks.best_lose
team_away.stats_home.streaks.best_goals_for
team_away.stats_home.streaks.best_goals_against
team_away.stats_away.played
team_away.stats_away.wins
team_away.stats_away.draws
team_away.stats_away.lose
team_away.stats_away.goal

In [27]:
league = '135'
season = '2019'

In [28]:
# League directory
dirName = league + '/' + season + '/' 

In [2]:
# Historicity: multi season directory
year_window = 1
start_season = 2019
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2019]


In [3]:
# Multi League directory
country = 'FR'
season = '2019'
dirCountry = '../' + country + '/'

In [4]:
# The last week played
curr_week = 28

In [30]:
df = pd.DataFrame()
standings = None
curr_standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings to week: team : stands
visited_standings = {}
# Easy access to last standings
current_standings = {}

# Main league
league = '61'
# Main
dirMain = dirCountry + league + '/' + str(season) + '/'

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if league == "teams.json":
        continue
    if ".csv" in league:
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'
            
            # Initializes all current standings
            if team_id not in current_standings:
                if team_id in ['85','95']:
                    json_standings = dirTeam + team_id + '_' + "27" + '.json'
                else:
                    json_standings = dirTeam + team_id + '_' + str(curr_week) + '.json'
                with open(json_standings) as f:
                    curr_standings = json.load(f)
                    current_standings[team_id] = curr_standings
            else:
                curr_standings = current_standings[team_id]
            
            # Gets standings from a team if they have not being added
            for week in list(range(1, 29, 1)):
                week = str(week)
                if week not in visited_standings:
                    visited_standings[week] = {}
                    if team_id not in visited_standings[week]:
                        try:
                            json_standings = dirTeam + team_id + '_' + week + '.json'
                            with open(json_standings) as f:
                                standings = json.load(f)
                                visited_standings[week][team_id] = standings
                        except:
                            visited_standings[week][team_id] = curr_standings
                            standings = curr_standings

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        w = fixture["week"]
                        if fixture["goals_away"] is None:
                            w = str(curr_week)
                        # Copa
                        if len(w) > 2:
                            w = str(curr_week)
                        # Gets standings from away_team
                        if away_team in visited_standings[w]:
                            away_team_stands = visited_standings[w][away_team]
                        else:
                            try:
                                if away_team in ['85','95']:
                                    fileAwayTeam = dirMain + away_team + '/' + away_team + '_' + "27" + '.json'
                                else:
                                    fileAwayTeam = dirMain + away_team + '/' + away_team + '_' + week + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                            except:
                                if away_team in ['85','95']:
                                    fileAwayTeam = dirMain + away_team + '/' + away_team + '_' + "27" + '.json'
                                else:
                                    fileAwayTeam = dirMain + away_team + '/' + away_team + '_' + str(curr_week) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                                    current_standings[away_team] = away_team_stands
                        # Gets away current standings if fixtures is set to null
                        if fixture["goals_away"] is None:
                            try:
                                away_team_stands = current_standings[away_team]
                                # Changes if game to be played
                                standings = current_standings[team_id]
                            except:
                                if away_team in ['85','95']:
                                    json_standings = dirMain + away_team + '/' + away_team + '_' + "27" + '.json'
                                else:
                                    json_standings = dirMain + away_team + '/' + away_team + '_' + str(curr_week) + '.json'
                                with open(json_standings) as f:
                                    curr_standings = json.load(f)
                                    current_standings[away_team] = curr_standings
                                away_team_stands = current_standings[away_team]
                                # Changes if game to be played
                                standings = current_standings[team_id]
                        else:
                            if away_team in visited_standings[w]:
                                away_team_stands = visited_standings[w][away_team]
                            else:
                                away_team_stands = current_standings[away_team]
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

In [31]:
df.head(10)

,id,week,goals_home,goals_away,league,season,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,...,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away
0,157671,26,0,0,61,2019,95,Strasbourg,11,38,...,1,39%,6,None,3,263.0,172.0,65%,NaN,NaN
0,157652,24,3,0,61,2019,95,Strasbourg,11,38,...,None,52%,1,None,1,418.0,338.0,81%,NaN,NaN
0,157596,19,2,1,61,2019,95,Strasbourg,11,38,...,1,60%,2,None,1,467.0,386.0,83%,NaN,NaN
0,157739,33,None,None,61,2019,95,Strasbourg,11,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,157629,22,1,2,61,2019,95,Strasbourg,11,38,...,4,56%,5,None,2,385.0,312.0,81%,NaN,NaN
0,157441,3,0,2,61,2019,95,Strasbourg,11,38,...,None,50%,2,None,3,395.0,321.0,81%,NaN,NaN
0,157489,8,1,0,61,2019,95,Strasbourg,11,38,...,2,54%,2,None,2,438.0,326.0,74%,NaN,NaN
0,157579,17,4,2,61,2019,95,Strasbourg,11,38,...,1,46%,1,None,3,395.0,321.0,81%,NaN,NaN
0,157452,4,2,2,61,2019,95,Strasbourg,11,38,...,2,41%,3,None,3,350.0,250.0,71%,NaN,NaN
0,157541,13,4,1,61,2019,95,Strasbourg,11,38,...,None,39%,2,1,6,281.0,196.0,70%,NaN,NaN


In [64]:
just_numbers = []
for fixture in visited_fixtures:
    just_numbers.append(int(fixture.strip('.json')))
print(just_numbers)

[2325877, 23288537, 23265614, 23274423, 23276825, 2325251, 23282531, 2325453, 23267716, 2325655, 23286234, 23281229, 23268517, 2326069, 23262611, 23283632, 23272721, 23270519, 23278427, 23277125, 2325271, 23280429, 23263612, 23279227, 23286735, 23284332, 2326099, 23268617, 23270719, 23266615, 23265914, 23289137, 23281530, 2325483, 23272621, 23275023, 2325685, 2325907, 23282631, 23278927, 2325897, 23274723, 2325675, 23262811, 2325473, 23273021, 23264613, 2325261, 23265814, 23287736, 23270619, 23289337, 23285434, 23281029, 23267916, 2326089, 23276425, 2326139, 2325735, 23263311, 2325533, 23277025, 23280729, 23267215, 2325937, 23269217, 23271219, 23279327, 23282431, 23265313, 23284733, 23289237, 23272421, 2325321, 23286935, 23275123, 23279127, 23271019, 2325513, 2325291, 2326119, 23274523, 2325715, 23284833, 23263111, 23268316, 23270218, 23272921, 2325917, 23281329, 23276725, 23283031, 23264913, 23285634, 23287536, 23261510, 2325746, 23278126, 23265514, 2325362, 23267616, 23263512, 232883

In [32]:
df.shape

(398, 124)

In [34]:
# csv to save
file_title = 'liguecopa_28.csv'
csv_file = dirCountry + file_title

In [35]:
df.to_csv(csv_file)

In [51]:
os.listdir(dirName)

['533',
 '798',
 '532',
 '.DS_Store',
 '543',
 '715',
 '529',
 '545',
 '542',
 '537',
 '530',
 '539',
 '538',
 '531',
 '536',
 '540',
 '541',
 '546',
 '548',
 '720',
 '727']